In [ ]:
## Celem tego miniprojektu jest gra, której zadaniem jest umieszczenie na planszy określonej liczby swoich pionków w rzędzie poziomo, pionowo lub ukośnie na planszy przed przeciwnikiem.
## Polega na wykonywaniu ruchu na zmianę z przeciwnikiem. Zasady są bliźniacze do gry w kółko i krzyżyk

In [ ]:
pip install kaggle-environments

In [ ]:
#Zdefiniowanie środowiska ConnectX do wersji 0.1.6
!pip install 'kaggle-environments>=0.1.6'



In [ ]:
#Tworzenie środowiska ConnectX
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render()

In [ ]:
#Tworzenie Agenta
#Aby utworzyć zgłoszenie, funkcja agenta powinna być w pełni hermetyzowana (bez zewnętrznych zależności).
#Gdy agent jest oceniany względem innych, nie będzie miał dostępu do obrazu dokowanego Kaggle.
#Ten agent wybiera niepustą kolumnę
def moj_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])



In [ ]:
#Testowanie agenta, tak jak jest to zamieszczone w poniższej animacji.
env.reset()
# Graj jako pierwszy agent przeciwko domyślnemu „losowemu” agentowi.
env.run([moj_agent, "random"])
env.render(mode="ipython", width=500, height=450)



In [ ]:
# Graj jako pierwsza pozycja przeciwko losowemu agentowi.
trainer = env.train([None, "random"])

observation = trainer.reset()

while not env.done:
    moj_ruch = moj_agent(observation, env.configuration)
    print("Mój ruch: ", moj_ruch)
    observation, reward, done, info = trainer.step(moj_ruch)
    # env.render(mode="ipython", width=100, height=90, header=False, controls=False)
env.render()

### Po wielokrotnym sprawdzeniu tej części kodu widać, że kompilator pokazuje różne wartości.



In [ ]:
#W tej sekcji odbywa się ocena agenta
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))

#Oszacowanie wydajności, przy pomocy 20 odcinków.
print("Mój agent przeciwko losowemu agentowi: ", mean_reward(evaluate("connectx", [moj_agent, "random"], num_episodes=20)))
print("Mój agent przeciwko losowemu agentowi: ", mean_reward(evaluate("connectx", [moj_agent, "negamax"], num_episodes=20)))

In [ ]:
#Kliknij dowolną kolumnę
env.play([None, "negamax"], width=500, height=450)



In [ ]:
##Zapis pliku "submission.py"
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(moj_agent, "submission.py")





<function moj_agent at 0x7ff3108b02f0> written to submission.py


In [ ]:
##Zatwierdzenie przeslania
import sys
out = sys.stdout
submission = utils.read_file("/kaggle/working/submission.py")
agent = utils.get_last_callable(submission)
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

